# Направите класификациони модел: Укусна азијска и индијска јела


## Класификатори кухиња 2

У овом другом часу о класификацији, истражићемо `више начина` за класификацију категоријских података. Такође ћемо научити о последицама избора једног класификатора уместо другог.

### [**Квиз пре предавања**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Предуслови**

Претпостављамо да сте завршили претходне лекције, јер ћемо наставити са неким концептима које смо раније научили.

За ову лекцију биће нам потребни следећи пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) је [збирка R пакета](https://www.tidyverse.org/packages) дизајнирана да учини науку о подацима бржом, лакшом и забавнијом!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) је [оквир](https://www.tidymodels.org/packages/) који обухвата пакете за моделирање и машинско учење.

-   `themis`: [themis пакет](https://themis.tidymodels.org/) пружа додатне кораке за рецепте који се баве небалансираним подацима.

Можете их инсталирати на следећи начин:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Алтернативно, скрипта испод проверава да ли имате потребне пакете за завршетак овог модула и инсталира их ако недостају.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Мапа класификације**

У нашој [претходној лекцији](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), покушали смо да одговоримо на питање: како да изаберемо између више модела? У великој мери, то зависи од карактеристика података и типа проблема који желимо да решимо (на пример, класификација или регресија?).

Раније смо научили о различитим опцијама које имате када класификујете податке користећи Microsoft-ов подсетник. Python-ов оквир за машинско учење, Scikit-learn, нуди сличан, али детаљнији подсетник који може додатно помоћи у сужењу избора проценитеља (други термин за класификаторе):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Савет: [погледајте ову мапу онлајн](https://scikit-learn.org/stable/tutorial/machine_learning_map/) и кликните дуж путање да бисте прочитали документацију.  
>  
> [Референтни сајт за Tidymodels](https://www.tidymodels.org/find/parsnip/#models) такође пружа одличну документацију о различитим типовима модела.

### **План** 🗺️

Ова мапа је веома корисна када имате јасно разумевање ваших података, јер можете „шетати“ дуж њених путања до доношења одлуке:

-   Имамо више од 50 узорака

-   Желимо да предвидимо категорију

-   Имамо означене податке

-   Имамо мање од 100.000 узорака

-   ✨ Можемо изабрати Linear SVC

-   Ако то не функционише, пошто имамо нумеричке податке

    -   Можемо пробати ✨ KNeighbors Classifier

        -   Ако ни то не функционише, пробајте ✨ SVC и ✨ Ensemble Classifiers

Ово је веома корисна путања коју треба пратити. Сада, хајде да одмах пређемо на то користећи [tidymodels](https://www.tidymodels.org/) оквир за моделирање: конзистентну и флексибилну колекцију R пакета развијених да подстакну добру статистичку праксу 😊.

## 2. Поделите податке и решите проблем неуравнотеженог скупа података.

Из претходних лекција смо научили да постоји скуп заједничких састојака у нашим кухињама. Такође, постојала је прилично неравномерна расподела у броју кухиња.

Ово ћемо решити на следећи начин:

-   Избацивањем најчешћих састојака који стварају конфузију између различитих кухиња, користећи `dplyr::select()`.

-   Коришћењем `recipe` који претходно обрађује податке како би их припремио за моделирање применом алгоритма `over-sampling`.

Ово смо већ обрадили у претходној лекцији, тако да би ово требало да буде лако 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Рад са неуравнотеженим подацима

Неуравнотежени подаци често имају негативан утицај на перформансе модела. Многи модели најбоље функционишу када је број опсервација једнак и, самим тим, имају потешкоћа са неуравнотеженим подацима.

Постоје два главна начина за рад са неуравнотеженим скуповима података:

-   додавање опсервација мањинској класи: `Прекомерно узорковање` (Over-sampling), на пример, коришћењем SMOTE алгоритма који синтетички генерише нове примере мањинске класе користећи најближе суседе тих случајева.

-   уклањање опсервација из већинске класе: `Потцењено узорковање` (Under-sampling)

У претходном часу, демонстрирали смо како да радимо са неуравнотеженим скуповима података користећи `рецепт`. Рецепт се може сматрати планом који описује које кораке треба применити на скуп података како би био спреман за анализу података. У нашем случају, желимо да имамо једнаку дистрибуцију броја наших кухиња за наш `тренинг скуп`. Хајде да почнемо.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Сада смо спремни да обучавамо моделе 👩‍💻👨‍💻!

## 3. Изван модела мултиномијалне регресије

У претходној лекцији, разматрали смо моделе мултиномијалне регресије. Хајде да истражимо неке флексибилније моделе за класификацију.

### Машине за подршку векторима

У контексту класификације, `Машине за подршку векторима` су техника машинског учења која настоји да пронађе *хиперплан* који "најбоље" раздваја класе. Хајде да погледамо једноставан пример:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ не раздваја класе. H2~ их раздваја, али само са малом маргином. H3~ их раздваја са максималном маргином.

#### Линеарни класификатор подршке векторима

Кластерисање подршке векторима (SVC) је део породице техника машинског учења заснованих на машинама подршке векторима. У SVC-у, хиперплан се бира тако да правилно раздвоји `већину` посматрања из тренинг скупа, али `може погрешно класификовати` нека посматрања. Дозвољавањем да неке тачке буду на погрешној страни, SVM постаје отпорнији на изузетке, чиме се побољшава генерализација на нове податке. Параметар који регулише ово кршење назива се `cost`, који има подразумевану вредност 1 (погледајте `help("svm_poly")`).

Хајде да направимо линеарни SVC тако што ћемо поставити `degree = 1` у полиномском SVM моделу.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Сада када смо обухватили кораке предобраде и спецификацију модела у *workflow*-у, можемо наставити са тренингом линеарног SVC-а и проценом резултата у истом процесу. За метрике перформанси, хајде да направимо сет метрика који ће процењивати: `тачност`, `осетљивост`, `позитивну предиктивну вредност` и `F меру`.

> `augment()` ће додати колону(е) за предикције у дате податке.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Машина за подршку вектора

Машина за подршку вектора (SVM) је проширење класификатора за подршку вектора како би се омогућила нелинеарна граница између класа. У суштини, SVM користи *трик са језгром* да прошири простор карактеристика и прилагоди се нелинеарним односима између класа. Једна популарна и изузетно флексибилна функција језгра коју SVM користи је *функција радијалне основе.* Хајде да видимо како ће се она показати на нашим подацима.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Много боље 🤩!

> ✅ Молимо вас да погледате:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Практично машинско учење са R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), Увод у статистичко учење са апликацијама у R
>
> за додатно читање.

### Класификатори најближих суседа

*K*-најближи сусед (KNN) је алгоритам у којем се свака опсервација предвиђа на основу њене *сличности* са другим опсервацијама.

Хајде да га применимо на наше податке.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Чини се да овај модел не ради баш најбоље. Вероватно ће промена аргумената модела (погледајте `help("nearest_neighbor")`) побољшати перформансе модела. Обавезно пробајте.

> ✅ Погледајте:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> да бисте сазнали више о класификаторима *K*-најближих суседа.

### Енсембл класификатори

Енсембл алгоритми функционишу тако што комбинују више основних проценитеља како би произвели оптималан модел, било кроз:

`bagging`: примену *функције просека* на колекцију основних модела

`boosting`: изградњу секвенце модела који се надовезују један на други ради побољшања предиктивних перформанси.

Хајде да започнемо са испробавањем модела Случајне шуме (Random Forest), који гради велику колекцију одлуковних стабала, а затим примењује функцију просека ради добијања бољег укупног модела.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Добар посао 👏!

Хајде да експериментишемо са моделом Boosted Tree.

Boosted Tree представља метод ансамбла који креира серију секвенцијалних одлука стабала где свако стабло зависи од резултата претходних стабала у настојању да постепено смањи грешку. Фокусира се на тежине погрешно класификованих ставки и прилагођава модел за следећи класификатор како би исправио грешке.

Постоје различити начини за подешавање овог модела (погледајте `help("boost_tree")`). У овом примеру, подешаваћемо Boosted стабла преко `xgboost` механизма.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Молимо вас да погледате:
>
> -   [Машинско учење за друштвене научнике](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Практично машинско учење са R](https://bradleyboehmke.github.io/HOML/)
>
> -   [Увод у статистичко учење са апликацијама у R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Истражује модел AdaBoost који је добра алтернатива за xgboost.
>
> за више информација о класификаторима ансамбла.

## 4. Додатно - поређење више модела

У овом лабораторијском раду смо применили прилично велики број модела 🙌. Може постати заморно или напорно креирати много радних токова из различитих сетова претпроцесора и/или спецификација модела, а затим израчунати метрике перформанси једну по једну.

Хајде да видимо да ли можемо да решимо ово креирањем функције која примењује листу радних токова на сет за обуку, а затим враћа метрике перформанси на основу тест сета. Користићемо `map()` и `map_dfr()` из пакета [purrr](https://purrr.tidyverse.org/) да применимо функције на сваки елемент у листи.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) функције вам омогућавају да замените многе for петље кодом који је и сажетији и лакши за читање. Најбоље место за учење о [`map()`](https://purrr.tidyverse.org/reference/map.html) функцијама је [поглавље о итерацији](http://r4ds.had.co.nz/iteration.html) у R за науку о подацима.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Пакет [**workflowset**](https://workflowsets.tidymodels.org/) омогућава корисницима да креирају и лако прилагоде велики број модела, али је углавном дизајниран за рад са техникама ресемплирања као што је `крстална валидација`, приступ који тек треба да обрадимо.

## **🚀Изазов**

Свака од ових техника има велики број параметара које можете прилагодити, на пример `cost` у SVM-у, `neighbors` у KNN-у, `mtry` (случајно изабрани предиктори) у Random Forest-у.

Истражите подразумеване вредности параметара за сваки модел и размислите шта би значило прилагођавање ових параметара за квалитет модела.

Да бисте сазнали више о одређеном моделу и његовим параметрима, користите: `help("model")`, на пример `help("rand_forest")`.

> У пракси, обично *процењујемо* *најбоље вредности* за ове параметре тако што тренирамо многе моделе на `симулираном скупу података` и меримо колико добро ти модели раде. Овај процес се назива **тјунинг**.

### [**Квиз након предавања**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Преглед и самостално учење**

У овим лекцијама има доста стручних термина, па одвојите минут да прегледате [ову листу](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) корисне терминологије!

#### ХВАЛА:

[`Allison Horst`](https://twitter.com/allison_horst/) за креирање невероватних илустрација које чине R приступачнијим и занимљивијим. Пронађите више илустрација у њеној [галерији](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) и [Jen Looper](https://www.twitter.com/jenlooper) за креирање оригиналне Python верзије овог модула ♥️

Срећно учење,

[Eric](https://twitter.com/ericntay), Златни амбасадор студената Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Илустрација од @allison_horst</figcaption>



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
